# Production scheduling -- the job shop problem

## Problem Description
- We have a number of jobs: Job 0, Job 1, Job 2, ...
- We have a number of machines: Machine 0, Machine 1, Machine 2, ...
- Each job must be processed on each machine in a specific order.
- for example: Job 0 must be processed on Machine 0, then on Machine 1, then on Machine 2.
- No machine can process more than one job at a time.
- We assume unlimited buffers between machines, so jobs can immediately move to the next machine if it's available.
- **Objective**: Minimize the total completion time (makespan), C_max.

the effect of the job shop problem is a schedule, frequentely presented as a Gantt chart.

# ![Job shop problem example](https://www.researchgate.net/publication/370490241/figure/fig1/AS:11431281155279843@1683131798236/Job-sequence-on-machines-for-a-job-shop-problem-with-3-machines-and-2-jobs.png)

The problem is NP-hard, the more jobs and machines, the harder it is to solve.



## Mini Problem Example

### Scenario Setup
- **Machines**: 0 and 1
- **Jobs**: 0 and 1
- Each job has two operations, one on each machine, but in different orders

### Job 0
- **Operation 1**: Machine 0 → Processing time = 5
- **Operation 2**: Machine 1 → Processing time = 6
- Thus, Job 0 must go first to Machine 0, then to Machine 1

### Job 1
- **Operation 1**: Machine 1 → Processing time = 4  
- **Operation 2**: Machine 0 → Processing time = 7
- Thus, Job 1 must go first to Machine 1, then to Machine 0

### Objective
Minimize the makespan (C_max)

In [6]:
!pip install pulp

import pulp

# 1) Create the problem
model = pulp.LpProblem("Small_Job_Shop", pulp.LpMinimize)

# 2) Decision variables
# Start times:
# s0_0: Start time of Job 0 on Machine 0
# s0_1: Start time of Job 0 on Machine 1
# s1_0: Start time of Job 1 on Machine 0
# s1_1: Start time of Job 1 on Machine 1
s0_0 = pulp.LpVariable("s0_0", lowBound=0, cat='Continuous')
s0_1 = pulp.LpVariable("s0_1", lowBound=0, cat='Continuous')
s1_0 = pulp.LpVariable("s1_0", lowBound=0, cat='Continuous')
s1_1 = pulp.LpVariable("s1_1", lowBound=0, cat='Continuous')

# Makespan
Cmax = pulp.LpVariable("Cmax", lowBound=0, cat='Continuous')

# 3) Processing times
# Job 0: M0=5, then M1=6
p0_m0 = 5 # processing time of Job 0 on Machine 0
p0_m1 = 6 # processing time of Job 0 on Machine 1

# Job 1: M1=4, then M0=7
p1_m1 = 4 # processing time of Job 1 on Machine 1
p1_m0 = 7 # processing time of Job 1 on Machine 0

# 4) Binary variables for machine usage ordering
# We need these to ensure no overlap on each machine - each machine can process only one job at a time.
# For Machine 0, let y0_01 = 1 if Job 0 finishes before Job 1 starts on M0, else 0.
# For Machine 1, let y1_01 = 1 if Job 0 finishes before Job 1 starts on M1, else 0.
y0_01 = pulp.LpVariable("y0_01", cat='Binary')  # ordering on Machine 0
y1_01 = pulp.LpVariable("y1_01", cat='Binary')  # ordering on Machine 1

# Define a "big M" large enough to cover maximum possible scheduling time
bigM = 9999

# 5) OBJECTIVE: minimize Cmax
model += Cmax, "Minimize_the_makespan"

# 6) CONSTRAINTS

# -- (A) Operation sequence for each job --

# Job 0 must finish on Machine 0 before starting on Machine 1:
model += s0_1 >= s0_0 + p0_m0, "Seq_Job0" # start time of Job 0 on Machine 1 must be greater than or equal to the start time of Job 0 on Machine 0 plus the processing time of Job 0 on Machine 0

# Job 1 must finish on Machine 1 before starting on Machine 0:
model += s1_0 >= s1_1 + p1_m1, "Seq_Job1" # start time of Job 1 on Machine 0 must be greater than or equal to the start time of Job 1 on Machine 1 plus the processing time of Job 1 on Machine 1

# -- (B) No overlap on Machine 0 --
# Machine 0 is used by:
#   - Job 0 (from s0_0 to s0_0 + p0_m0)
#   - Job 1 (from s1_0 to s1_0 + p1_m0)
# Use a big-M approach with binary y0_01:

# If y0_01 = 1, Job0 finishes before Job1 starts on M0:
#     s1_0 >= s0_0 + p0_m0 - bigM*(1 - y0_01)
model += s1_0 >= s0_0 + p0_m0 - bigM*(1 - y0_01), "NoOverlap0a"

# If y0_01 = 0, Job1 finishes before Job0 starts on M0:
#     s0_0 >= s1_0 + p1_m0 - bigM*(y0_01)
model += s0_0 >= s1_0 + p1_m0 - bigM*(y0_01), "NoOverlap0b"

# -- (C) No overlap on Machine 1 --
# Machine 1 is used by:
#   - Job 0 (from s0_1 to s0_1 + p0_m1)
#   - Job 1 (from s1_1 to s1_1 + p1_m1)
# Use a big-M approach with y1_01:

# If y1_01 = 1, Job0 finishes before Job1 starts on M1:
#     s1_1 >= s0_1 + p0_m1 - bigM*(1 - y1_01)
model += s1_1 >= s0_1 + p0_m1 - bigM*(1 - y1_01), "NoOverlap1a"

# If y1_01 = 0, Job1 finishes before Job0 starts on M1:
#     s0_1 >= s1_1 + p1_m1 - bigM*(y1_01)
model += s0_1 >= s1_1 + p1_m1 - bigM*(y1_01), "NoOverlap1b"

# -- (D) Makespan definition --
# Finish time for Job 0 = time it completes on Machine 1 = s0_1 + p0_m1
# Finish time for Job 1 = time it completes on Machine 0 = s1_0 + p1_m0

model += Cmax >= s0_1 + p0_m1, "Makespan_Job0"
model += Cmax >= s1_0 + p1_m0, "Makespan_Job1"

# 7) Solve the model
model.solve(pulp.PULP_CBC_CMD(msg=False))

# 8) Print results
print("Status:", pulp.LpStatus[model.status])
print("Objective (makespan) =", pulp.value(model.objective))

# Decision variable values:
print("--- Start Times ---")
print("s0_0 (Job0 on M0) =", pulp.value(s0_0))
print("s0_1 (Job0 on M1) =", pulp.value(s0_1))
print("s1_0 (Job1 on M0) =", pulp.value(s1_0))
print("s1_1 (Job1 on M1) =", pulp.value(s1_1))

print("--- Binary Orders ---")
print("y0_01 (Machine 0 order) =", pulp.value(y0_01))
print("y1_01 (Machine 1 order) =", pulp.value(y1_01))

Status: Optimal
Objective (makespan) = 12.0
--- Start Times ---
s0_0 (Job0 on M0) = 0.0
s0_1 (Job0 on M1) = 5.0
s1_0 (Job1 on M0) = 5.0
s1_1 (Job1 on M1) = 0.0
--- Binary Orders ---
y0_01 (Machine 0 order) = 1.0
y1_01 (Machine 1 order) = 0.0


In [7]:
import matplotlib.pyplot as plt

# Create Gantt chart
fig, ax = plt.subplots(figsize=(10, 6))

# Colors for each job
colors = ['#FF9999', '#66B2FF']

# For each machine
for m in range(2):
    # Plot Job 0 on this machine
    if m == 0:
        start = pulp.value(s0_0)
        duration = p0_m0
    else:
        start = pulp.value(s0_1)
        duration = p0_m1
    ax.barh(m, duration, left=start, color=colors[0], label='Job 0' if m==0 else "")

    # Plot Job 1 on this machine
    if m == 0:
        start = pulp.value(s1_0)
        duration = p1_m0
    else:
        start = pulp.value(s1_1)
        duration = p1_m1
    ax.barh(m, duration, left=start, color=colors[1], label='Job 1' if m==0 else "")

# Customize the chart
ax.set_yticks(range(2))
ax.set_yticklabels(['Machine 0', 'Machine 1'])
ax.set_xlabel('Time')
ax.set_title('Job Shop Schedule')
ax.grid(True)
ax.legend()

plt.tight_layout()
plt.show()


# Excercise 1

Play with the code above to see how the schedule changes with different processing times.



#More general job shop problem example

## Data Setup
Let's define for each job:
- The machine sequence the job must follow
- The processing time on each machine (in hours)

### Job Specifications:

**Job 0:**
- Machine sequence: Machine 0 → Machine 1 → Machine 2
- Processing times: 5, 6, 4 hours respectively

**Job 1:**
- Machine sequence: Machine 1 → Machine 2 → Machine 0  
- Processing times: 4, 3, 7 hours respectively

**Job 2:**
- Machine sequence: Machine 0 → Machine 2 → Machine 1
- Processing times: 6, 4, 5 hours respectively

We will store this data in two parallel 2D lists:
- `machine_order`: Contains the sequence of machines for each job
- `processing_time`: Contains the corresponding processing times



In [8]:
#!pip install pulp

import pulp

# -------------------------
# 1) PROBLEM PARAMETERS
# -------------------------
num_jobs = 3
num_machines = 4

# Kolejność maszyn dla każdej pracy
machine_order = [
    [0, 1, 2, 3],  # Job 0
    [1, 2, 3, 0],  # Job 1
    [0, 3, 2, 1]   # Job 2
]

# Czas przetwarzania dla każdej operacji w kolejności maszyn
processing_time = [
    [5, 6, 4 ,3],  # Job 0
    [4, 3, 7, 6],  # Job 1
    [6, 4, 5, 7]   # Job 2
]

# -------------------------
# 2) DECISION VARIABLES
# -------------------------
# start_time[m, j] = czas rozpoczęcia Job j na maszynie m
start_time = pulp.LpVariable.dicts(
    "start_time",
    [(m, j) for m in range(num_machines) for j in range(num_jobs)],
    lowBound=0,
    cat='Continuous'
)

# Cmax = czas zakończenia ostatniej operacji (makespan)
Cmax = pulp.LpVariable('Cmax', lowBound=0, cat='Continuous')

# -------------------------
# 3) MODEL
# -------------------------
model = pulp.LpProblem("Job_Shop_Scheduling", pulp.LpMinimize)

# Funkcja celu: minimalizacja makespanu
model += Cmax, "Minimize_makespan"

# -------------------------
# 4) CONSTRAINTS
# -------------------------

# (A) Ograniczenia kolejności operacji w każdej pracy
for j in range(num_jobs):
    for op in range(num_machines - 1):
        current_machine = machine_order[j][op]
        next_machine = machine_order[j][op + 1]

        current_op_time = processing_time[j][op]

        # Następna operacja musi rozpocząć się po zakończeniu poprzedniej
        model += start_time[next_machine, j] >= (
            start_time[current_machine, j] + current_op_time
        ), f"SequenceConstraint_Job{j}_Op{op}"

# (B) Ograniczenia niekolizyjne na maszynach (big-M)
bigM = 9999

# zmienne binarne do określenia kolejności między parami zadań na tej samej maszynie
y = pulp.LpVariable.dicts(
    "y",
    [(m, j1, j2) for m in range(num_machines) for j1 in range(num_jobs) for j2 in range(num_jobs) if j1 != j2],
    cat='Binary'
)

for m in range(num_machines):
    for j1 in range(num_jobs):
        for j2 in range(num_jobs):
            if j1 != j2:
                # Indeksy operacji dla maszyn
                op_j1 = machine_order[j1].index(m)
                proc_j1 = processing_time[j1][op_j1]

                op_j2 = machine_order[j2].index(m)
                proc_j2 = processing_time[j2][op_j2]

                # Ograniczenia niekolizyjne: jeśli y=1 -> j1 przed j2, jeśli y=0 -> j2 przed j1
                model += start_time[m, j2] >= start_time[m, j1] + proc_j1 - bigM * (1 - y[m, j1, j2])
                model += start_time[m, j1] >= start_time[m, j2] + proc_j2 - bigM * (y[m, j1, j2])

# (C) Definicja makespanu
for j in range(num_jobs):
    last_machine = machine_order[j][-1]
    last_op_idx = num_machines - 1
    finish_time_j = start_time[last_machine, j] + processing_time[j][last_op_idx]
    model += Cmax >= finish_time_j, f"CmaxConstraint_Job{j}"

# -------------------------
# 5) SOLVE
# -------------------------
model.solve(pulp.PULP_CBC_CMD(msg=False))

print("Solution Status:", pulp.LpStatus[model.status])
print("Objective (makespan) =", pulp.value(model.objective))

# Wyświetlenie czasów rozpoczęcia każdej operacji
for (m, j) in start_time:
    print(f"Start time of Job {j} on Machine {m} = {pulp.value(start_time[m, j])}")


Solution Status: Optimal
Objective (makespan) = 24.0
Start time of Job 0 on Machine 0 = 6.0
Start time of Job 1 on Machine 0 = 17.0
Start time of Job 2 on Machine 0 = 0.0
Start time of Job 0 on Machine 1 = 11.0
Start time of Job 1 on Machine 1 = 0.0
Start time of Job 2 on Machine 1 = 17.0
Start time of Job 0 on Machine 2 = 17.0
Start time of Job 1 on Machine 2 = 4.0
Start time of Job 2 on Machine 2 = 10.0
Start time of Job 0 on Machine 3 = 21.0
Start time of Job 1 on Machine 3 = 10.0
Start time of Job 2 on Machine 3 = 6.0


In [9]:
# Create Gantt chart
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import colorsys

# --- Ustawienia wykresu ---
fig_height = max(6, num_machines * 0.5)  # dynamiczna wysokość w zależności od liczby maszyn
fig, ax = plt.subplots(figsize=(12, fig_height))

# --- Generowanie kolorów dla jobów ---
def generate_colors(n):
    colors = []
    for i in range(n):
        hue = i / n
        saturation = 0.7
        value = 0.9
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        colors.append(rgb)
    return colors

colors = generate_colors(num_jobs)

# --- Tworzenie prostokątów dla każdej operacji ---
for m in range(num_machines):
    for j in range(num_jobs):
        # Pobierz czas startu i czas trwania operacji
        start = pulp.value(start_time[m, j])
        op_idx = machine_order[j].index(m)
        duration = processing_time[j][op_idx]

        # Wysokość prostokąta w zależności od liczby maszyn
        rect_height = min(0.8, 0.95 - (0.01 * num_machines))

        # Utworzenie prostokąta
        rect = patches.Rectangle(
            (start, m),         # pozycja (x, y)
            duration,           # szerokość
            rect_height,        # wysokość
            facecolor=colors[j],
            edgecolor='black',
            alpha=0.7
        )
        ax.add_patch(rect)

        # Rozmiar czcionki dynamiczny w zależności od liczby jobów i maszyn
        text_size = max(8, min(10, 200/(num_jobs * num_machines)))

        # Dodanie numeru jobu na środku prostokąta
        ax.text(start + duration/2, m + rect_height/2, f'J{j}',
                horizontalalignment='center',
                verticalalignment='center',
                fontsize=text_size)

# --- Dostosowanie osi i etykiet ---
ax.set_ylim(-0.5, num_machines - 0.5)
ax.set_xlim(0, pulp.value(Cmax) * 1.05)  # 5% marginesu

# Etykiety osi Y (maszyny) i rozmiar ticków dynamiczny
tick_size = max(8, min(10, 120/num_machines))
ax.set_yticks(range(num_machines))
ax.set_yticklabels([f'Machine {m}' for m in range(num_machines)], fontsize=tick_size)
ax.set_xlabel('Time', fontsize=tick_size)
ax.grid(True, axis='x', linestyle='--', alpha=0.7)

# Tytuł wykresu
plt.title('Job Shop Schedule - Gantt Chart', pad=20, fontsize=tick_size + 2)
plt.tight_layout()
plt.show()


22ms->35ms

# Excercise 1b:
Play with the code above, see how the Cmax changes with different processing times, different variations of the problem.
# Excercise 2

Add a machine to the problem. Use %%timeit to see how the solver performance changes.

# Excercise 3:

Make the problem interactive, add a way to specify the number of jobs and machines interactively

In [25]:
%%timeit

import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import pulp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import colorsys

# --- Helper function: generowanie unikalnych kolorów dla jobów ---
def generate_colors(n):
    colors = []
    for i in range(n):
        hue = i / n
        saturation = 0.7
        value = 0.9
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        colors.append(rgb)
    return colors

# --- Suwaki do wyboru liczby zadań i maszyn ---
num_jobs_slider = widgets.IntSlider(min=2, max=5, step=1, value=5, description='Liczba Zadań:')
num_machines_slider = widgets.IntSlider(min=2, max=5, step=1, value=5, description='Liczba Maszyn:')

# --- Widget wyjścia do wyświetlania wyników dynamicznie ---
output = widgets.Output()
display(num_jobs_slider, num_machines_slider, output)

# --- Funkcja rozwiązująca problem i rysująca wykres Gantta ---
def solve_and_plot_interactive(num_jobs, num_machines):
    with output:
        clear_output(wait=True)
        print(f"Rozwiązywanie dla {num_jobs} Zadań i {num_machines} Maszyn...")

        # --- 1. Generowanie dynamicznych danych problemu ---
        machine_order = []
        processing_time = []
        for j in range(num_jobs):
            # Losowa permutacja maszyn dla każdego zadania
            job_machines = list(range(num_machines))
            random.shuffle(job_machines)
            machine_order.append(job_machines)
            # Losowe czasy przetwarzania operacji (1-10)
            processing_time.append([random.randint(1, 10) for _ in range(num_machines)])

        print("Wygenerowana kolejność maszyn (machine_order):", machine_order)
        print("Wygenerowane czasy przetwarzania (processing_time):", processing_time)

        # --- 2. Zmienne decyzyjne ---
        start_time = pulp.LpVariable.dicts(
            "start_time",
            [(m, j) for m in range(num_machines) for j in range(num_jobs)],
            lowBound=0,
            cat='Continuous'
        )
        Cmax = pulp.LpVariable('Cmax', lowBound=0, cat='Continuous')

        # --- 3. Model ---
        model = pulp.LpProblem("Job_Shop_Scheduling_Interactive", pulp.LpMinimize)
        model += Cmax, "Minimize_makespan"

        # --- 4A. Ograniczenia kolejności operacji ---
        for j in range(num_jobs):
            for op in range(num_machines - 1):
                current_machine_idx = machine_order[j][op]
                next_machine_idx = machine_order[j][op + 1]
                current_op_time = processing_time[j][op]

                model += start_time[next_machine_idx, j] >= (
                    start_time[current_machine_idx, j] + current_op_time
                ), f"SequenceConstraint_Job{j}_Op{op}"

        # --- 4B. Brak nakładania się na tej samej maszynie (Big-M) ---
        bigM = sum(sum(pt_list) for pt_list in processing_time) + 1
        if bigM < 1000: bigM = 1000  # minimalna wartość Big-M

        y = pulp.LpVariable.dicts(
            "y",
            [(m, j1, j2) for m in range(num_machines) for j1 in range(num_jobs) for j2 in range(num_jobs) if j1 != j2],
            cat='Binary'
        )

        for m in range(num_machines):
            for j1 in range(num_jobs):
                for j2 in range(num_jobs):
                    if j1 != j2:
                        op_j1 = machine_order[j1].index(m)
                        proc_j1 = processing_time[j1][op_j1]
                        op_j2 = machine_order[j2].index(m)
                        proc_j2 = processing_time[j2][op_j2]

                        model += start_time[m, j2] >= start_time[m, j1] + proc_j1 - bigM * (1 - y[m, j1, j2]), f"NoOverlap_M{m}_J{j1}_J{j2}_a"
                        model += start_time[m, j1] >= start_time[m, j2] + proc_j2 - bigM * y[m, j1, j2], f"NoOverlap_M{m}_J{j1}_J{j2}_b"

        # --- 4C. Definicja makespanu Cmax ---
        for j in range(num_jobs):
            last_op_on_machine = machine_order[j][-1]
            last_op_time = processing_time[j][num_machines - 1]
            model += Cmax >= start_time[last_op_on_machine, j] + last_op_time, f"CmaxConstraint_Job{j}"

        # --- 5. Rozwiązanie modelu ---
        model.solve(pulp.PULP_CBC_CMD(msg=False))

        print("\nStatus Rozwiązania:", pulp.LpStatus[model.status])
        print("Cel (makespan) =", pulp.value(model.objective))

        # --- 6. Rysowanie wykresu Gantta ---
        fig_height = max(6, num_machines * 0.75)
        fig, ax = plt.subplots(figsize=(12, fig_height))
        job_colors = generate_colors(num_jobs)

        for m in range(num_machines):
            for j in range(num_jobs):
                start_val = pulp.value(start_time[m, j])
                op_idx = machine_order[j].index(m)
                duration_val = processing_time[j][op_idx]
                rect_height = min(0.8, 0.95 - (0.01 * num_machines))

                rect = patches.Rectangle(
                    (start_val, m),
                    duration_val,
                    rect_height,
                    facecolor=job_colors[j],
                    edgecolor='black',
                    alpha=0.7
                )
                ax.add_patch(rect)

                text_size = max(8, min(10, 200/(num_jobs * num_machines)))
                ax.text(start_val + duration_val/2, m + rect_height/2, f'J{j}',
                        horizontalalignment='center',
                        verticalalignment='center',
                        fontsize=text_size)

        ax.set_ylim(-0.5, num_machines - 0.5)
        ax.set_xlim(0, pulp.value(Cmax) * 1.05)
        tick_size = max(8, min(10, 120/num_machines))
        ax.set_yticks(range(num_machines))
        ax.set_yticklabels([f'Maszyna {m}' for m in range(num_machines)], fontsize=tick_size)
        ax.set_xlabel('Czas', fontsize=tick_size)
        ax.grid(True, axis='x', linestyle='--', alpha=0.7)
        plt.title('Harmonogram produkcji - Wykres Gantta (Interaktywny)', pad=20, fontsize=tick_size + 2)
        plt.tight_layout()
        plt.show()

# --- Połączenie suwaki z funkcją za pomocą interactive_output ---
widgets.interactive_output(solve_and_plot_interactive, {'num_jobs': num_jobs_slider, 'num_machines': num_machines_slider});


IntSlider(value=5, description='Liczba Zadań:', max=5, min=2)

IntSlider(value=5, description='Liczba Maszyn:', max=5, min=2)

Output()

IntSlider(value=5, description='Liczba Zadań:', max=5, min=2)

IntSlider(value=5, description='Liczba Maszyn:', max=5, min=2)

...

Output()

IntSlider(value=5, description='Liczba Zadań:', max=5, min=2)

IntSlider(value=5, description='Liczba Maszyn:', max=5, min=2)

Output()

855 ms ± 349 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Czasy:

Lmaszyn = 2, Lzadań = 2, Czas = 269 ms ± 56.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Lmaszyn = 4, Lzadań = 3, Czas = 280 ms ± 71.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Lmaszyn = 5, Lzadań = 5, Czas = 775 ms ± 459 ms per loop (mean ± std. dev. of 7 runs, 1 loop each), 855 ms ± 349 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) -  duża różnica w wykonaniu pętli wynikająca ze zwiększonej ilości zmiennych i rozwiązań


The problem can be very hard for a larger number of jobs and machines. There are some heuristics that can be used to solve it. Some interesting methods are:
- Genetic algorithms
- Simulated annealing
- Tabu search
- Ant colony optimization
- Particle swarm optimization
